In [3]:
import os
import shutil
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import time
import cv2

# Fix CPU core detection issue
os.environ["LOKY_MAX_CPU_COUNT"] = str(os.cpu_count())

# Create transform for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Siamese Network Definition
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # Remove the final classification layer
        
    def forward(self, x):
        return self.cnn(x)

# Load pre-trained Siamese network
siamese_net = SiameseNetwork()
siamese_net.eval()

def convert_svg_to_png(svg_path, png_path=None):
    """Convert an SVG file to a PNG file."""
    if png_path is None:
        # Create a temporary PNG file path if not provided
        png_path = os.path.splitext(svg_path)[0] + ".png"

    try:
        from wand.image import Image as WandImage  # Alternative using Wand
        with WandImage(filename=svg_path, resolution=300) as img:
            img.format = 'png'
            img.save(filename=png_path)
            print(f"Converted {svg_path} to PNG using Wand")
    except ImportError:
        print("Wand not installed, trying svgpathtools instead.")

        try:
            # Alternative using svgpathtools
            import svgpathtools
            paths, attributes = svgpathtools.svg2paths(svg_path)
            img = Image.new("RGBA", (512, 512), (255, 255, 255, 0))  # Transparent background
            img.save(png_path)
            print(f"Converted {svg_path} to PNG using svgpathtools")
        except ImportError:
            print("Both Wand and svgpathtools are not installed. Cannot convert SVG.")
            return None

    return png_path

def compute_color_histogram(img):
    """Compute color histogram features from an image."""
    # Convert PIL image to cv2 format
    img_cv = np.array(img)
    img_cv = img_cv[:, :, ::-1].copy()  # RGB to BGR
    
    # Convert to HSV colorspace
    img_hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
    
    # Calculate histograms
    h_hist = cv2.calcHist([img_hsv], [0], None, [32], [0, 180])
    s_hist = cv2.calcHist([img_hsv], [1], None, [32], [0, 256])
    v_hist = cv2.calcHist([img_hsv], [2], None, [32], [0, 256])
    
    # Normalize histograms
    h_hist = cv2.normalize(h_hist, h_hist, 0, 1, cv2.NORM_MINMAX)
    s_hist = cv2.normalize(s_hist, s_hist, 0, 1, cv2.NORM_MINMAX)
    v_hist = cv2.normalize(v_hist, v_hist, 0, 1, cv2.NORM_MINMAX)
    
    # Combine histograms
    color_hist = np.concatenate([h_hist, s_hist, v_hist]).flatten()
    return color_hist

def compute_edge_histogram(img):
    """Compute edge histogram for shape analysis."""
    # Convert PIL image to cv2 format
    img_cv = np.array(img)
    if img_cv.shape[2] == 4:  # RGBA
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGBA2BGR)
    else:
        img_cv = img_cv[:, :, ::-1].copy()  # RGB to BGR
    
    # Convert to grayscale
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Calculate edge direction histogram
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    
    # Create histogram of edge directions
    hist = np.zeros(36)  # 10-degree bins
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i, j] > 0:  # If it's an edge pixel
                bin_idx = int(angle[i, j] / 10) % 36
                hist[bin_idx] += 1
    
    # Normalize histogram
    if np.sum(hist) > 0:
        hist = hist / np.sum(hist)
    
    return hist

def load_and_preprocess_image(image_path):
    """Load an image, convert to tensor, and preprocess."""
    try:
        if image_path.lower().endswith('.svg'):
            # Convert SVG to temporary PNG file
            temp_png_path = convert_svg_to_png(image_path)
            if temp_png_path is None:
                return None, None, None
                
            # Load the converted PNG
            img = Image.open(temp_png_path).convert("RGB")
            
            # Clean up the temporary file after loading
            try:
                os.remove(temp_png_path)
            except:
                pass  # Ignore deletion errors
        else:
            img = Image.open(image_path).convert("RGB")
        
        # Compute additional features
        color_hist = compute_color_histogram(img)
        edge_hist = compute_edge_histogram(img)
        
        # Transform for CNN
        img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
        
        return img_tensor, color_hist, edge_hist
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None, None

def extract_features(image_paths):
    """Extract image features using the Siamese network and additional color/shape analysis."""
    cnn_features = []
    color_features = []
    shape_features = []
    valid_paths = []
    
    for image_path in tqdm(image_paths, desc="Extracting Features", unit="image"):
        img_tensor, color_hist, edge_hist = load_and_preprocess_image(image_path)
        if img_tensor is not None:
            with torch.no_grad():
                cnn_feature = siamese_net(img_tensor).squeeze().numpy()
                # Normalize the feature vector to unit length for better cosine similarity
                cnn_feature = cnn_feature / np.linalg.norm(cnn_feature)
            
            cnn_features.append(cnn_feature)
            color_features.append(color_hist)
            shape_features.append(edge_hist)
            valid_paths.append(image_path)
    
    print(f"Successfully extracted features from {len(valid_paths)} images out of {len(image_paths)} total images")
    return np.array(cnn_features), np.array(color_features), np.array(shape_features), valid_paths

def compute_similarity_matrices(cnn_features, color_features, shape_features):
    """Compute similarity matrices for different feature types."""
    # CNN feature similarity (cosine)
    cnn_sim = np.dot(cnn_features, cnn_features.T)
    
    # Color histogram similarity (correlation)
    color_sim = np.zeros((len(color_features), len(color_features)))
    for i in range(len(color_features)):
        for j in range(i, len(color_features)):
            color_corr = np.corrcoef(color_features[i], color_features[j])[0, 1]
            color_sim[i, j] = color_sim[j, i] = color_corr
    
    # Shape feature similarity (correlation)
    shape_sim = np.zeros((len(shape_features), len(shape_features)))
    for i in range(len(shape_features)):
        for j in range(i, len(shape_features)):
            shape_corr = np.corrcoef(shape_features[i], shape_features[j])[0, 1]
            shape_sim[i, j] = shape_sim[j, i] = shape_corr
    
    print(f"Computed similarity matrices of shape {cnn_sim.shape}")
    return cnn_sim, color_sim, shape_sim

def combined_similarity(cnn_sim, color_sim, shape_sim, cnn_weight=0.5, color_weight=0.3, shape_weight=0.2):
    """Combine multiple similarity matrices with weights."""
    # Handle NaN values by replacing with 0
    color_sim = np.nan_to_num(color_sim)
    shape_sim = np.nan_to_num(shape_sim)
    
    # Combine similarities with weights
    combined = (cnn_weight * cnn_sim) + (color_weight * color_sim) + (shape_weight * shape_sim)
    return combined

def hierarchical_clustering(similarity_matrix, valid_paths, similarity_threshold=0.9):
    """Group similar logos using hierarchical agglomerative clustering."""
    n = len(valid_paths)
    print(f"Starting hierarchical clustering for {n} images with similarity threshold {similarity_threshold}")
    
    # Initialize each logo as its own cluster
    clusters = [{i} for i in range(n)]
    
    # Convert similarity to distance
    distance_matrix = 1 - similarity_matrix
    
    # Initialize progress bar for the maximum possible iterations (n-1 merges at most)
    max_iterations = n - 1
    pbar = tqdm(total=max_iterations, desc="Clustering Progress")
    
    # Track the number of completed iterations
    iterations = 0
    
    # While there are clusters that can be merged
    while len(clusters) > 1 and iterations < max_iterations:
        min_distance = float('inf')
        merge_indices = (-1, -1)
        
        # Find the closest pair of clusters
        pairs_checked = 0
        total_pairs = (len(clusters) * (len(clusters) - 1)) // 2
        
        for i in range(len(clusters)):
            for j in range(i+1, len(clusters)):
                # Compute average linkage distance between clusters
                total_dist = 0
                count = 0
                for idx1 in clusters[i]:
                    for idx2 in clusters[j]:
                        total_dist += distance_matrix[idx1, idx2]
                        count += 1
                
                pairs_checked += 1
                if pairs_checked % 100 == 0 or pairs_checked == total_pairs:
                    print(f"\rChecking cluster pairs: {pairs_checked}/{total_pairs}", end="")
                
                if count > 0:
                    avg_dist = total_dist / count
                    if avg_dist < min_distance and avg_dist < (1 - similarity_threshold):
                        min_distance = avg_dist
                        merge_indices = (i, j)
        
        print()  # New line after progress output
        
        # If no more clusters can be merged, break
        if merge_indices[0] == -1 or min_distance >= (1 - similarity_threshold):
            print(f"No more clusters can be merged (min distance: {min_distance:.4f}, threshold: {1-similarity_threshold:.4f})")
            break
        
        # Merge the clusters
        i, j = merge_indices
        print(f"Merging clusters {i} (size {len(clusters[i])}) and {j} (size {len(clusters[j])}) with distance {min_distance:.4f}")
        clusters[i] = clusters[i].union(clusters[j])
        clusters.pop(j)
        
        iterations += 1
        pbar.update(1)
        
        # Print current clustering status every few iterations
        if iterations % 5 == 0 or iterations == max_iterations:
            cluster_sizes = [len(c) for c in clusters]
            print(f"Current state: {len(clusters)} clusters, sizes: {cluster_sizes}")
    
    pbar.close()
    
    # Convert to the expected format
    groups = {}
    for i, cluster in enumerate(clusters):
        if len(cluster) > 1:  # Only groups with at least 2 logos
            groups[i] = [valid_paths[idx] for idx in cluster]
            print(f"Group {i}: {len(groups[i])} logos")
    
    # Handle unique logos (those that aren't in any group)
    grouped_indices = set().union(*[cluster for cluster in clusters if len(cluster) > 1]) if groups else set()
    unique_logos = []
    for i in range(n):
        if i not in grouped_indices:
            unique_logos.append(valid_paths[i])
    
    if unique_logos:
        groups[-1] = unique_logos
        print(f"Unique logos: {len(unique_logos)}")
    
    return groups

def save_logo_groups(groups, output_folder="logo_groups"):
    """Save logo groups into folders."""
    if os.path.exists(output_folder):
        print(f"Removing existing output folder: {output_folder}")
        shutil.rmtree(output_folder)
    
    os.makedirs(output_folder)
    print(f"Created output folder: {output_folder}")
    
    unique_counter = 1
    group_count = 0
    total_logos = 0
    
    for label, logos in tqdm(groups.items(), desc="Saving Logo Groups"):
        if label == -1:
            # Unique logos (not similar to any other)
            for logo_path in logos:
                unique_folder = os.path.join(output_folder, f"unique_logo_{unique_counter}")
                os.makedirs(unique_folder, exist_ok=True)
                dest_path = os.path.join(unique_folder, os.path.basename(logo_path))
                shutil.copy(logo_path, dest_path)
                print(f"Copied unique logo to {dest_path}")
                unique_counter += 1
                total_logos += 1
        else:
            # Similar logos
            group_folder = os.path.join(output_folder, f"group_{label}")
            os.makedirs(group_folder, exist_ok=True)
            print(f"Created group folder: {group_folder}")
            
            for logo_path in logos:
                dest_path = os.path.join(group_folder, os.path.basename(logo_path))
                shutil.copy(logo_path, dest_path)
                print(f"Copied similar logo to {dest_path}")
                total_logos += 1
            
            group_count += 1
    
    print(f"✅ Saved {group_count} groups and {unique_counter-1} unique logos (total: {total_logos} logos) into {output_folder}.")

def main():
    """Main function to process logos and group them by similarity."""
    start_time = time.time()
    
    logo_folder = "C:/Users/bawsu/Desktop/Python-test/test/logo"
    output_folder = "logo_groupz"
    
    # Get all image paths
    image_extensions = (".png", ".jpg", ".jpeg", ".svg")
    image_paths = [os.path.join(logo_folder, f) for f in os.listdir(logo_folder) 
                  if f.lower().endswith(image_extensions)]
    
    print(f"Found {len(image_paths)} image files in {logo_folder}")
    
    # Extract features using the Siamese network and additional analysis
    print("🔍 Extracting logo features...")
    cnn_features, color_features, shape_features, valid_paths = extract_features(image_paths)
    
    if len(cnn_features) == 0:
        print("No valid images found for processing.")
        return
    
    # Compute similarity matrices
    print("🔍 Computing logo similarities...")
    cnn_sim, color_sim, shape_sim = compute_similarity_matrices(cnn_features, color_features, shape_features)
    
    # Combine similarity matrices
    print("🔍 Combining similarity measures...")
    combined_sim = combined_similarity(cnn_sim, color_sim, shape_sim)
    
    # Group similar logos using hierarchical clustering with higher threshold
    print("🔍 Grouping similar logos...")
    # Higher threshold (0.92) for more stringent matching
    groups = hierarchical_clustering(combined_sim, valid_paths, similarity_threshold=0.92)
    
    # Save logo groups into folders
    print("💾 Saving logo groups to folders...")
    save_logo_groups(groups, output_folder)
    
    # Print summary
    total_groups = sum(1 for label in groups if label != -1)
    total_unique = len(groups.get(-1, [])) if -1 in groups else 0
    print(f"📊 Summary: Found {total_groups} groups of similar logos and {total_unique} unique logos.")
    print(f"⏱️ Total execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

C:\Users\bawsu\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Found 2053 image files in C:/Users/bawsu/Desktop/Python-test/test/logo
🔍 Extracting logo features...


Extracting Features:  16%|████████▋                                              | 326/2053 [00:13<01:03, 27.29image/s]

Wand not installed, trying svgpathtools instead.


Extracting Features:  16%|█████████                                              | 336/2053 [00:13<01:09, 24.82image/s]

...

Copied similar logo to logo_groupz\group_832\subaru.com.tr - logo.png
Copied similar logo to logo_groupz\group_832\subaru.es - logo.png
Copied similar logo to logo_groupz\group_832\subaru.jp - logo.png
Copied similar logo to logo_groupz\group_832\subaru.nc - logo.png
Copied similar logo to logo_groupz\group_832\subaruvilnius.lt - logo.png
Created group folder: logo_groupz\group_846
Copied similar logo to logo_groupz\group_846\synergie.com - logo.png
Copied similar logo to logo_groupz\group_846\synergie.es - logo.jpg
Copied similar logo to logo_groupz\group_846\synergiejobs.nl - logo.png
Created group folder: logo_groupz\group_849
Copied similar logo to logo_groupz\group_849\synlab.by - logo.png
Copied similar logo to logo_groupz\group_849\synlab.hr - logo.png
Copied similar logo to logo_groupz\group_849\synlab.si - logo.png
Created group folder: logo_groupz\group_862
Copied similar logo to logo_groupz\group_862\tbwa.it - logo.png
Copied similar logo to logo_groupz\group_862\tbwa.co.nz 

Saving Logo Groups:  98%|██████████████████████████████████████████████████████████▉ | 171/174 [00:02<00:00, 86.91it/s]

Copied similar logo to logo_groupz\group_943\trulynolenalagoas.com.br - logo.png
Copied similar logo to logo_groupz\group_943\trulynolenbarueri.com.br - logo.png
Created group folder: logo_groupz\group_957
Copied similar logo to logo_groupz\group_957\vbh.cz - logo.png
Copied similar logo to logo_groupz\group_957\vbh.hr - logo.png
Copied similar logo to logo_groupz\group_957\vbh.ro - logo.png
Copied similar logo to logo_groupz\group_957\vbhtrgovina.si - logo.png
Created group folder: logo_groupz\group_971
Copied similar logo to logo_groupz\group_971\vkf-renzel.bg - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.co.uk - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.cz - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.dk - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.fr - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.hu - logo.png
Copied similar logo to logo_groupz\group_971\vkf-renzel.nl - logo.p

Saving Logo Groups: 100%|████████████████████████████████████████████████████████████| 174/174 [00:04<00:00, 43.19it/s]

Copied unique logo to logo_groupz\unique_logo_831\wurth.ua - logo.png
Copied unique logo to logo_groupz\unique_logo_832\wurthsaudi.com - logo.png
Copied unique logo to logo_groupz\unique_logo_833\wwf.at - logo.png
Copied unique logo to logo_groupz\unique_logo_834\wwf.or.th - logo.png
Copied unique logo to logo_groupz\unique_logo_835\wwf.ro - logo.jpg
Copied unique logo to logo_groupz\unique_logo_836\xtransfer.cn - logo.png
Copied unique logo to logo_groupz\unique_logo_837\xwiftracingevents.be - logo.png
Copied unique logo to logo_groupz\unique_logo_838\ymca-cnw.org - logo.png
Copied unique logo to logo_groupz\unique_logo_839\ymcachicago.org - logo.png
Copied unique logo to logo_groupz\unique_logo_840\ymcatrivalley.org - logo.png
Copied unique logo to logo_groupz\unique_logo_841\yonyou.com.hk - logo.png
Copied unique logo to logo_groupz\unique_logo_842\yord.pl - logo.png
Copied unique logo to logo_groupz\unique_logo_843\yves-rocher.pt - logo.jpg
Copied unique logo to logo_groupz\unique_